In [1]:
%pwd

'd:\\vit\\langchain\\End-to-End-Medical-Chat-Bot-Generative-AI\\research'

In [2]:
import os
os.chdir("../") #go one folder back

In [3]:
%pwd

'd:\\vit\\langchain\\End-to-End-Medical-Chat-Bot-Generative-AI'

In [4]:
from langchain.document_loaders import PyPDFLoader , DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

def load_pdf_file(data):
    loader = DirectoryLoader(data ,
                            glob = "*.pdf" ,
                            loader_cls = PyPDFLoader)
    documents = loader.load()
    return documents

In [5]:
extracted_data = load_pdf_file(data = "Data/")

In [6]:
# extracted_data

In [7]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500,
                                                    chunk_overlap = 30)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [8]:
text_chunks = text_split(extracted_data)
print("length of text_chunks" , len(text_chunks))

length of text_chunks 7034


In [9]:
from langchain.embeddings import HuggingFaceEmbeddings
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [10]:
embeddings = download_hugging_face_embeddings()

C:\Users\yogit\AppData\Local\Temp\ipykernel_27312\1637511200.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")
c:\Users\yogit\anaconda3\envs\llmapp\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


c:\Users\yogit\anaconda3\envs\llmapp\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\yogit\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [11]:
query_result = embeddings.embed_query("hello world")
print("length",len(query_result))

length 384


In [17]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os
from dotenv import load_dotenv

load_dotenv()
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
pc = Pinecone(api_key = os.environ.get('PINECONE_API_KEY') )
index_name = "medibot"

pc.create_index(
    name = index_name,
    dimension=384,
    metric="cosine",
    spec = ServerlessSpec(
        cloud = "aws",
        region = "us-east-1"
    )
)

In [18]:
from langchain_pinecone import PineconeVectorStore
#Embed each chunk and upsert the embeddings into your Pinecone Index
docsearch = PineconeVectorStore.from_documents(
    documents = text_chunks,
    index_name = index_name,
    embedding=embeddings
)

In [19]:
#load existing index
from langchain_pinecone import PineconeVectorStore
#embed each chunk and upsert the embeddings into your Pinecone index
docserach = PineconeVectorStore.from_existing_index(
    index_name = index_name,
    embedding = embeddings
)


Caching the list of root modules, please wait!
Caching the list of root modules, please wait!
(This will only be done once - type '%rehashx' to reset cache!)


(This will only be done once - type '%rehashx' to reset cache!)


Caching the list of root modules, please wait!
(This will only be done once - type '%rehashx' to reset cache!)


Caching the list of root modules, please wait!
(This will only be done once - type '%rehashx' to reset cache!)



c:\Users\yogit\anaconda3\envs\llmapp\lib\site-packages\IPython\core\completerlib.py:150: UserWarning: This is now an optional IPython functionality, setting rootmodules_cache requires you to install the `pickleshare` library.
  ip.db['rootmodules_cache'] = rootmodules_cache


In [20]:
docsearch

In [24]:
retriever = docsearch.as_retriever(search_type = "similarity" , search_kwargs = {"k" : 7})

In [25]:
retrieved_docs = retriever.invoke("What is Acne?")

In [26]:
retrieved_docs

[Document(id='8641ae4b-f443-4874-9fdf-7c70400b1bb9', metadata={'page': 37.0, 'source': 'Data\\Gale Encyclopedia of Medicine. Vol. 1. 2nd ed.pdf'}, page_content='Acid reflux seeHeartburn\nAcidosis seeRespiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nDefinition\nAcne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when thepores of the skin become clogged with oil, dead skincells, and bacteria.\nDescription\nAcne vulgaris, the medical term for common acne, is'),
 Document(id='4d84c8a1-f5a6-466d-a656-2f1cdc47b954', metadata={'page': 38.0, 'source': 'Data\\Gale Encyclopedia of Medicine. Vol. 1. 2nd ed.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25Acne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceousglands become inflamed. (Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)GEM - 0001 to 0432 - A  10/22/03 1:41 PM  P

In [ ]:
from langchain_groq import ChatGroq
GROQ_API_KEY = os.environ.get('GROQ_API_KEY')
os.environ['GROQ_API_KEY'] = GROQ_API_KEY
llm = ChatGroq( model_name = "Llama3-8b-8192",
                temperature = 0.4)

In [47]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant frp question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know, Use minimum three maximum seven or eight sentences maximum and keep the answer concise. "
    "\n\n"
    "{context}"
)

human_prompt = "{input}"

# Using strings to define the messages directly in the template
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", human_prompt)
])

In [48]:
question_answer_chain = create_stuff_documents_chain(llm , prompt)
rag_chain = create_retrieval_chain(retriever , question_answer_chain)

response = rag_chain.invoke({"input":"how to cure depression ?"})
print(response["answer"])

According to the provided context, depression can be treated with a combination of professional treatment, family support, and lifestyle changes. Some effective treatments for depression include:

1. Antidepressant medications such as desipramine (Norpramin), imipramine (Tofranil), and fluoxetine (Prozac).
2. Short-term psychotherapy, which can be combined with medications.
3. Lifestyle changes such as maintaining a calm environment, avoiding overstimulation, getting plenty of rest, regular exercise, and a proper diet.
4. Chinese herbs, which may help soften mood swings.
5. Biofeedback, which can help patients control symptoms such as irritability, poor self-control, racing thoughts, and sleep problems.
6. A diet low in vanadium and high in vitamin C, which may be helpful in reducing depression.

It's also important to note that outside help, such as nursing homes and governmental assistance, may be necessary for some families to relieve stress and provide support.
